# Programming project 2: Harmonic Vibrational analysis

In this project you will be doing a harmonic vibrational analysis on water, benzene and 3-chloro-1-butene. All the necessary input files present in the `input` directory, and you can check your answers with the numerical solutions in the `solutions` folder.

## Step 1: Read the coordinate data

You can use 'Step 1' from Project 1 to read in your coordinate data.

_(Bonus: Can you write a small Python library that provides this functionality so it can be reused?)_

## Step 2: Read the cartesian Hessian data

In the directory there are also some `*_hessian.txt` files, which contain a Hessian matrix, which consists of second derivatives of the energy with respect to two atomic positions:

$$F_{ij} = \frac{\partial^2 V}{\partial q_i \partial q_j} \, .$$

The first integer is the number of atoms of the system and the rest of the lines have the following format:

$$\begin{array}{ccc} F_{x_1,x_1} & F_{x_1,y_1} & F_{x_1,z_1} \\ F_{x_1,x_2} & F_{x_1,y_2} & F_{x_1,z_2} \\ & \\ \ldots & \ldots & \dots \\ & \\ F_{x_2,x_1} & F_{x_2,y_1} & F_{x_2,z_1} \\ & \\ \ldots & \ldots & \ldots \end{array} \, .$$

Note that the units of this Hessian matrix are given as $\text{Hartree} \, \text{bohr}^{-2}$.

## Step 3: Mass-weigh the Hessian matrix

The mass-weighted Hessian matrix is calculated by dividing each element of the Hessian matrix by the product of the square roots of the masses of the atoms associated with the given coordinates:

$$F^M_{ij} = \frac{F_{ij}}{\sqrt{m_i m_j}}$$

Like the previous exercise, we are working in amu.


## Step 4: Diagonalize the mass-weighted Hessian matrix

In this step, you should calculate the eigenvalues of the previously calculated mass-weighted Hessian:

$$\mathbf{F}^M \mathbf{L} = \mathbf{L} \Lambda \, .$$

## Step 5: Compute the harmonic vibrational frequencies

Finally, the vibrational frequencies are proportional to the square root of the eigenvalues of the mass-weighted Hessian:

$$\omega_i = \text{constant} \,  \sqrt{\lambda_i} \, .$$

In the solution, frequencies are reported in $\text{cm}^{-1}$.